In [19]:
import pandas as pd
import numpy as np
import pickle
import re
import unicodedata

In [20]:

data=pd.read_csv("../merged/len3_ndb_blp_DN_firm.csv",encoding="utf-8",index_col=0)
data.loc[data["メーカー名"].isna()].shape
data.loc[data["メーカー名"].isna(),"医薬品名"].value_counts()

#n2n
name2name_ii = pd.read_pickle('./pickle/name2name_ii.pkl')
name2name_nk = pd.read_pickle('./pickle/name2name_nk.pkl')
name2name_cl_nk=pd.read_pickle('./pickle/name2name_cl_nk.pkl')
name2name=pd.read_pickle('./pickle/name2name.pkl')

#c2f
code2firm=pd.read_pickle('./pickle/code2firm.pkl')
# n2f
name2firm_ii = pd.read_pickle('./pickle/name2firm_ii.pkl')
name2firm_nk = pd.read_pickle('./pickle/name2firm_nk.pkl')
name2firm_ma=pd.read_pickle('./pickle/name2firm_ma.pkl')
name2firm=pd.read_pickle('./pickle/name2firm.pkl')

# cl nk
cleaned2firm=pd.read_pickle('./pickle/cleaned2firm.pkl')
name2cleaned=pd.read_pickle('./pickle/name2cleaned.pkl')

# name2firm_cl_nk = {name: cleaned2firm[cleaned] for name, cleaned in name2cleaned.items()}
# update 
name2name_nk.update(name2name)
name2name_nk.update(name2name_ii)
name2name_nk.update(name2name_cl_nk)

name2firm_nk.update(name2firm)
name2firm_nk.update(name2firm_ii)
name2firm_nk.update(cleaned2firm)

# make them list
name2name_ii=list(name2name_ii.items())
name2name_nk=list(name2name_nk.items())
name2name_cl_nk=list(name2name_cl_nk.items())
len(name2name_ii),len(name2name_nk)


C:\Users\hakos\AppData\Local\Temp\ipykernel_37936\3859624651.py:1: DtypeWarning: Columns (118,119,131) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len3_ndb_blp_DN_firm.csv",encoding="utf-8",index_col=0)


(134, 396)

In [21]:
name2name_cl_nk

[('イトラコナゾール50mgカプセル', 'イトラコナゾールカプセル50mg「SW」'),
 ('フェルビナクテープ35mg「三笠」 ', 'フェルビナクテープ35mg「三笠」'),
 ('イトラコナゾール50mg錠', 'イトラコナゾール錠50「MEEK」'),
 ('イソコナゾール硝酸塩300mg腟錠', 'イソコナゾール硝酸塩腟錠300mg「F」'),
 ('テルビナフィン塩酸塩125mg錠', 'テルビー錠125mg'),
 ('ジクロフェナクナトリウム50mg坐剤', 'ジクロフェナクナトリウム坐剤50mg「JG」'),
 ('ラクツロース経口ゼリー分包16.05g「サトウ」 ', 'ラクツロース経口ゼリー分包16.05g「サトウ」'),
 ('ベクロメタゾンプロピオン酸エステル8.5mg8.5g点鼻液', 'ナイスピー点鼻液50μg'),
 ('ロペラミド塩酸塩1mgカプセル', 'ロペナカプセル1mg'),
 ('トルナフタート液 ', 'ハイアラージン外用液2%'),
 ('ベクロメタゾン点鼻液50μg「DSP」 ', 'ベクロメタゾン点鼻液50μg「DSP」'),
 ('塩酸ロペラミド1mg錠', 'カグダリンカプセル1mg'),
 ('ロキソプロフェンナトリウム60mg錠', 'ロキソプロフェンナトリウム錠60mg「クニヒロ」'),
 ('スルファメトキサゾール・トリメトプリム錠', 'ダイフェン配合錠'),
 ('亜鉛華軟膏', '亜鉛華軟膏「コザカイ・M」'),
 ('イトラコナゾール100mg錠', 'イトラコナゾール錠100「MEEK」'),
 ('ビクロックス錠400 ', 'ビクロックス錠400'),
 ('クロモグリク酸ナトリウム190mg9.5mL点鼻液', 'クロモグリク酸Na点鼻液2%「ファイザー」'),
 ('アシクロビン錠200 ', 'アシクロビン錠200'),
 ('ツロブテロール塩酸塩1mg錠', 'セキナリン錠1mg'),
 ('ケトプロフェンパップXR120mg「テイコク」 ', 'ケトプロフェンパップXR120mg「テイコク」'),
 ('ロペラミド塩酸塩1mg錠', 'ロペラミド錠1mg「EMEC」'),
 ('ベクロメタゾン点鼻液50μg「杏林」 ', 'ベクロメタゾン点鼻液50μg「杏林」'),
 ('

In [22]:
cnt=0
def delete_ok(name2name,delete_no=0):
    delete_list=[]
    # keyとvalueについて、「」内が同じかどうかを確認する
    for i,(key,value) in enumerate(name2name):
        cleaned_key = unicodedata.normalize('NFKC', re.sub(r'\u3000', ' ', re.search(r'.*\u3000', key).group()) if re.search(r'.*\u3000', key) else key)
        #空白を削除
        cleaned_key = cleaned_key.replace(' ','')
        # print(cleaned_key,value,cleaned_key==value)
        if cleaned_key==value or key==value:
            # cnt+=1
            delete_list.append(i)
            continue
        try:
            pattern = r'「(.*?)」'
            key_v=re.findall(pattern, key)[0]
            key_v = ''.join([unicodedata.normalize('NFKC', char) for char in key_v])
            value_v=re.findall(pattern, value)[0]
            value_v = ''.join([unicodedata.normalize('NFKC', char) for char in value_v])
            # print(key_v,value_v,key_v==value_v)
            
            if key_v==value_v:
                delete_list.append(i)
        except:
            continue
            # remove key and value
    # delete
    if delete_no>0:
        name2name=[name2name[i] for i in range(len(name2name)) if i in delete_list]
    else:
        name2name = [name2name[i] for i in range(len(name2name)) if i not in delete_list]
    return name2name
name2name_ii=delete_ok(name2name_ii)
name2name_nk=delete_ok(name2name_nk,delete_no=1)
name2name_cl_nk=delete_ok(name2name_cl_nk)
len(name2name_ii),len(name2name_nk),len(name2name_cl_nk)

(6, 310, 34)

In [23]:
for x in dict(name2name_ii).keys():
    print(x,data.loc[data["医薬品名"]==x,"医薬品コード"].unique().astype(int))

亜鉛華軟膏 [662640446]
複方ヨード・グリセリン [662310022]
イオウ [662650006]
薬用炭 [612370089]
乳酸 [612370088]
ジスチグミン臭化物錠　５ｍｇ [611230132]


In [24]:
for x in dict(name2name_nk).keys():
    print(x,data.loc[data["医薬品名"]==x,"医薬品コード"].unique().astype(int))

フラボキサート塩酸塩錠２００ｍｇ「ＹＤ」 [620564332]
クロピドグレル錠７５ｍｇ「ファイザー」 [622416301]
アシクロビル錠２００ｍｇ「ファイザー」 [621353002]
フロラーズ点鼻液５０μｇ１１２噴霧用　８．１６ｍｇ１６ｍＬ [622297601]
シロドシン錠４ｍｇ「ＫＮ」 [622686301]
ベラプロストＮａ錠２０μｇ「ファイザー」 [622147201]
ビダラビン軟膏３％「ＭＥＥＫ」 [620006495]
ムコソレートＬカプセル４５　４５ｍｇ [610463199]
クロピドグレル錠７５ｍｇ「ＫＮ」 [622433401]
エピナスチン塩酸塩錠２０ｍｇ「ファイザー」 [622088201]
イトラコナゾール錠１００「ＭＥＥＫ」　１００ｍｇ [620001959]
オフロキシン眼軟膏０．３％ [620003030]
タムスロシン塩酸塩ＯＤ錠０．１ｍｇ「ファイザー」 [622206501]
セキナリン錠１ｍｇ [620005017]
シメチジン錠２００ｍｇ「トーワ」 [620429716]
ラクツロースシロップ６５％「武田テバ」 [620881504]
タムスロシン塩酸塩ＯＤ錠０．２ｍｇ「ファイザー」 [622206601]
トニール錠１０μｇ [620006996]
ミリカレット点鼻液５０μｇ２８噴霧用　２．０４ｍｇ４ｍＬ [620004274]
ジクロフェナクＮａ坐剤２５ｍｇ「武田テバ」 [621362242]
ビフォノール外用液１％ [621373520]
プロカテロール塩酸塩錠５０μｇ「日医工」　０．０５ｍｇ [621957602]
ラグノスゼリー分包１６．０５ｇ　４０．４９６％ [620881701]
アデフロニック錠２５ｍｇ [620004439]
プロカテロール塩酸塩錠５０μｇ「サワイ」　０．０５ｍｇ [621369711]
カサールクリーム３％ [620006404]
ラスプジン錠１ｍｇ [620004392]
アポラキート錠２００ｍｇ [620004377]
エンテカビル錠０．５ｍｇ「ＤＳＥＰ」 [622535001]
シンバスタチン錠５ｍｇ「アメル」 [620000052]
マリンゾールクリーム１％ [620006533]
トアラセット配合錠「ファイザー」 [622648701]
塩酸アンブロキソール内用液０

In [25]:
name2name_nk=dict(name2name_nk)
name2firm = {key: name2firm_nk[key] for key in name2firm_nk if key in name2name_nk}
len(name2firm)

310

In [26]:
data["医薬品コード"]=data["医薬品コード"].astype(int).astype(str)
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm)
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm_ma)
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2cleaned).map(name2firm)
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品コード"].map(code2firm)
data_lack=data[data["メーカー名"].isna()]
print(len(set(data_lack["医薬品名"])))
data.to_csv("../merged/len3_ndb_blp_DN_firm_val.csv",encoding="utf-8")

47


医薬品名に対してレセプト電算コードが一意でないもの

In [27]:

# data=pd.read_csv("../merged/len3_ndb_blp_DN.csv",encoding="shift-jis",index_col=0)
data["医薬品コード"]=data["医薬品コード"].astype(int)
name_set=set(data["医薬品名"])
for name in name_set:
    code_set=set(data.loc[data["医薬品名"]==name,"医薬品コード"])
    # yjコードが変わってるのは結構ある
    yj_code_set=set(data.loc[data["医薬品名"]==name,"薬価基準収載医薬品コード"])
    if len(code_set)!=1:
        print(name,code_set,yj_code_set)

ジクロフェナクナトリウム５０ｍｇ坐剤 {622330800, 622849300} {'1147700J3017'}
クロトリマゾール１％クリーム {622330200, 622780800} {'2655700N1010'}
ビホナゾール１％クリーム {622331600, 622781000} {'2655708N1016'}


統一だろ


- ビホナゾール１％クリーム {622331600, 622781000}

- クロトリマゾール１％クリーム {622330200, 622780800} クロトリマゾールクリーム1％「イワキ」岩城製薬

- ジクロフェナクナトリウム５０ｍｇ坐剤 {622330800, 622849300}

In [214]:
data[data["医薬品コード"]==622330200]["year"]
data[data["医薬品コード"]==622780800]["year"]

21095    2020
24583    2021
Name: year, dtype: int64

In [48]:
data.loc[data["メーカー名"].isna()].shape
data.loc[data["メーカー名"].isna(),"医薬品名"].value_counts()

ジスチグミン臭化物錠　５ｍｇ                     16
Ｄソルビトール                            16
複方ヨード・グリセリン                        15
トルナフタート液　２％                        14
薬用炭                                14
                                   ..
ケトチフェン点鼻液０．０５％「ツルハラ」　６．０４８ｍｇ８ｍＬ     1
クロモグリク酸ナトリウム１９０ｍｇ９．５ｍＬ点鼻液           1
ビホナゾール１％液                           1
カモスタットメシル酸塩１００ｍｇ錠                   1
イトラコナゾール５０ｍｇ錠                       1
Name: 医薬品名, Length: 61, dtype: int64